## Configuration

In [99]:
### Configuration
# Import dependencies
import os
from collections import Counter
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [ ]:
# Authenticate to Spotify
spotify, userID, user_token = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

## Collect artists from playlist

In [82]:
# Read in favorite new artists from culled ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
# Get all the artistIDs in that playlist
artistIDs = fetchArtistsFromPlaylist(winnersPlaylist)
# Sort and deduplicate artists by value counts (get value count dict; sort this dict)
artistIDs = list(dict(sorted(Counter(artistIDs).items(), key=lambda x:x[1], reverse=True)).keys())
artistIDs

['1GmsPCcpKgF9OhlNXjOsbS', '0QWrMNukfcVOmgEU0FEDyD', '5jQsMNuCW0iZeFTz6YUK8K']

## Get the recent albums for each artist

In [88]:
# Find recent songs < 3 years old from new artists
recentAlbums = fetchRecentAlbums(artistIDs, maxAge=3)
recentAlbums

['33cj3kzLqVOg9zvy69Wrc8',
 '63ButSDtUefzCmdE3AXdor',
 '1mu38xw1WDRabnfwlvGqNO',
 '7H7nlUOKxq1u3Ev2TP5ib5',
 '5u8SP8bbxcA7zJoAusBfgc',
 '0G9S31E4tXEPAdLTR6Os3M',
 '2OAeKCwp9IyZou5eFsiSa0',
 '68L5xVV9wydotfDXEik7eD']

## Get all the tracks on these playlists

In [93]:
def fetchTracks(albumIDs):
    with spotify.token_as(user_token):
        # Cut the df into chunks
        n = 0
        trackURIs = []
        while n < len(albumIDs):
            albumsToGet = albumIDs[n:(n+20)]
            try:
                    # Query spotify for that album's tracks
                    nestedList = [[y.uri for y in x.tracks.items] for x in spotify.albums(albumsToGet, market='from_token')]
                    # Unnest the list 
                    tracks = [item for sublist in nestedList for item in sublist]
                    trackURIs += tracks
                    # Wait 2 seconds so we do not exceed our API call allowance
                    time.sleep(2)
            except:
                    # If there is an error in the batch, go through each separately
                    for albumID in albumsToGet:
                        try:
                            nestedList = [[y.uri for y in x.tracks.items] for x in spotify.album(albumID, market='from_token')]
                            # Unnest the list 
                            tracks = [item for sublist in nestedList for item in sublist]
                            trackURIs += tracks
                            # Wait 2 seconds so we do not exceed our API call allowance
                            time.sleep(2)
                        except: 
                            pass
            finally:
                n += 20
    # Drop duplicates
    trackURIs = trackURIs.unique()
    return trackURIs

In [94]:
recentTracks = fetchTracks(recentAlbums)
recentTracks

AttributeError: 'list' object has no attribute 'unique'

## Add recent albums to a playlist

In [92]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore Further')
# Add songs to the playlist
addToPlaylist(recentTracks, playlistID)

In [ ]:
# Cull both newFaves and oldFaves into CurrentYearSongCandidates